In [0]:
%run /Workspace/Users/mrudula.r@diggibyte.com/Databricks-Assignment/source_to_bronze/utils
     

In [0]:
from pyspark.sql.functions import avg, count, desc, col
emp_df = spark.read.table("assignment.assignment_resources.employee")
emp_df = emp_df.withColumn("Salary", col("Salary").cast("double"))

salary_df = emp_df.groupBy("DepartmentID") \
    .sum("Salary") \
    .withColumnRenamed("sum(Salary)", "total_salary") \
    .orderBy(desc("total_salary"))

salary_df = add_at_load_date(salary_df)
salary_df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("assignment.assignment_resources.salary_by_dept")


dept_df = spark.read.option("header", True).csv("/Volumes/assignment/assignment_resources/source/Department-Q1.csv")
country_df = spark.read.option("header", True).csv("/Volumes/assignment/assignment_resources/source/Country-Q1 (1).csv")

if "CountryID" in emp_df.columns and "DepartmentID" in emp_df.columns:
    emp_count = emp_df.groupBy("DepartmentID", "CountryID") \
        .agg(count("*").alias("emp_count"))

    emp_count = add_at_load_date(emp_count)

    emp_count.write.format("delta") \
        .mode("overwrite") \
        .saveAsTable("assignment.assignment_resources.emp_count")

avg_age = emp_df.groupBy("DepartmentID") \
    .agg(avg("Age").alias("avg_age"))

avg_age = add_at_load_date(avg_age)

avg_age.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("assignment.assignment_resources.avg_age")

     